### Import library

In [1]:
import os
os.chdir('../..')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
import glob
import re

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [3]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_i751gywdpfc2VSKWRM8YHtF8DuMz

 


In [4]:
ace.check_session_timeout(s)

14394.241388

## Alpha Mixing Templates

In [18]:
'''
signal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));

signal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));

signal1 * (1 + signal2)
'''

'\nsignal1 = scale_down(<cs_operator>(ts_backfill(<data1>,126)));\n\nsignal2 = scale_down(<cs_operator>(ts_backfill(<data2>,126)));\n\nsignal1 * (1 + signal2)\n'

## Read all passed alphas

In [ ]:
#os.chdir('Results/signal')

In [21]:
file_direc = glob.glob('*')
file_direc

['A_signalform.csv',
 'B_signalform.csv',
 'C_signalform.csv',
 'D_signalform.csv',
 'pre_signalform.csv',
 'Sep03GLB_signalform.csv',
 'Sep13GLB_signalform.csv']

In [22]:
expressions = {}
for file in file_direc:
    expressions[file.split('_')[0]] = pd.read_csv(file)['expression']
#expressions

In [23]:
expressions.keys()

dict_keys(['A', 'B', 'C', 'D', 'pre', 'Sep03GLB', 'Sep13GLB'])

In [24]:
expressions['Sep13GLB']

0     s1_Sep13GLB_0 = ts_zscore (ts_backfill (vec_av...
1     s1_Sep13GLB_1 = ts_zscore (ts_backfill (vec_av...
2     s1_Sep13GLB_2 = ts_zscore (ts_backfill (vec_av...
3     s1_Sep13GLB_3 = ts_zscore (ts_backfill (vec_av...
4     s1_Sep13GLB_4 = ts_zscore (ts_backfill (vec_av...
5     s1_Sep13GLB_5 = ts_zscore (ts_backfill (vec_av...
6     s1_Sep13GLB_6 = ts_zscore (ts_backfill (vec_av...
7     s1_Sep13GLB_7 = ts_zscore (ts_backfill (vec_av...
8     s1_Sep13GLB_8 = ts_zscore (ts_backfill (vec_av...
9     s1_Sep13GLB_9 = ts_zscore (ts_backfill (vec_av...
10    s1_Sep13GLB_10 = ts_zscore (ts_backfill (vec_a...
11    s1_Sep13GLB_11 = ts_zscore (ts_backfill (vec_a...
12    s1_Sep13GLB_12 = ts_zscore (ts_backfill (vec_a...
13    s1_Sep13GLB_13 = ts_zscore (ts_backfill (vec_a...
14    s1_Sep13GLB_14 = ts_zscore (ts_backfill (vec_a...
15    s1_Sep13GLB_15 = ts_zscore (ts_backfill (vec_a...
16    s1_Sep13GLB_16 = ts_zscore (ts_backfill (vec_a...
17    s1_Sep13GLB_17 = ts_zscore (ts_backfill (v

## Template A & Sep13GLB mixing
use top 32 results

In [59]:
#region= "GLB", universe = "MINVOL1M", neutralization = 'MARKET', truncation = 0.01

In [25]:
A = expressions['A'][:32]
Sep13GLB = expressions['Sep13GLB']

In [26]:
alpha_list_ASep13GLB = []

for i in range(3):
    for j in range(3):
        alpha_list_ASep13GLB += [ace.generate_alpha(f'{A[i]};{Sep13GLB[j]};add(scale_down(0.7*signal_A_{i}), scale_down(signal_Sep13GLB_{j}), filter = false)', region= "GLB", universe = "MINVOL1M", neutralization = x, truncation = 0.005, decay = 7) for x in ['COUNTRY', 'MARKET']]

alpha_list_ASep13GLB

[{'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'GLB',
   'universe': 'MINVOL1M',
   'delay': 1,
   'decay': 7,
   'neutralization': 'COUNTRY',
   'truncation': 0.005,
   'pasteurization': 'ON',
   'testPeriod': 'P0Y0M0D',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': 'pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore(ts_backfill(anl69_cps_best_eeps_nxt_yr, 21),21);factor_A_0 = group_zscore(ts_ir(returns, 252), industry);alpha_A_0 = regression_neut(stock_A_0, factor_A_0);signal_A_0 = trade_when(pro_A_0<=0.5, alpha_A_0, pro_A_0>0.5);s1_Sep13GLB_0 = ts_zscore (ts_backfill (vec_avg(mdl139_score), 252), 252);s2_Sep13GLB_0 = ts_zscore (ts_backfill (vec_avg(oth193_shield2), 252), 252);group_Sep13GLB_0 = pv13_20_minvol_1m_sector;signal_Sep13GLB_0 = group_zscore(s1_Sep13GLB_0+s2_Sep13GLB_0, group_Sep13GLB_0);add(scale_down(0.7*signal_A_0), scale_down(signal_Sep13GLB_0), filter = false)'},
 {'t

In [27]:
result = ace.simulate_alpha_list_multi(s, alpha_list_ASep13GLB)

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [17:11<00:00, 171.87s/it]


In [28]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,13011550,20000000,1598,1607,0.2205,0.1256,0.0272,0.001140,3.18,4.21,2012-01-22,3RreA26,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,FAIL,PASS
1,12713043,20000000,1599,1606,0.2192,0.1228,0.0264,0.001120,3.14,4.19,2012-01-22,P3aO1oM,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,FAIL,PASS
2,12766640,20000000,1522,1532,0.2259,0.1233,0.0328,0.001092,2.84,3.84,2012-01-22,APZ738Y,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
3,12830448,20000000,981,1000,0.2409,0.1239,0.0309,0.001029,2.65,3.70,2012-01-22,XnQ8pka,pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
4,12573082,20000000,982,999,0.2397,0.1214,0.0305,0.001013,2.61,3.67,2012-01-22,d0VRKA2,pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
5,13019486,20000000,979,1002,0.2400,0.1257,0.0337,0.001047,2.57,3.55,2012-01-22,3RrejMz,pro_A_2 = oth460_1l_ccns;stock_A_2 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
6,12787538,20000000,981,1001,0.2391,0.1235,0.0333,0.001033,2.53,3.52,2012-01-22,8Q9evom,pro_A_2 = oth460_1l_ccns;stock_A_2 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
7,12855572,20000000,937,959,0.2460,0.1241,0.0347,0.001009,2.47,3.48,2012-01-22,rlgPoWj,pro_A_1 = oth460_1l_ccns;stock_A_1 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
8,12988715,20000000,935,962,0.2444,0.1254,0.0389,0.001026,2.38,3.32,2012-01-22,j0RrezQ,pro_A_2 = oth460_1l_ccns;stock_A_2 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
9,13786182,20000000,1600,1607,0.2115,0.1331,0.0532,0.001259,2.33,2.94,2012-01-22,VPj38NG,pro_A_0 = oth460_1l_acda;stock_A_0 = ts_zscore...,PASS,PASS,PASS,PASS,PASS,PASS,PASS


In [29]:
result_st1.to_csv('A&Sep16GLB_mixing_add.csv')

In [30]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df.head()

,alpha_id,endDate,limit,name,result,startDate,themes,value,year
0,n2R8WAa,NaN,1.58,LOW_SHARPE,PASS,NaN,NaN,2.6400,NaN
1,n2R8WAa,NaN,1.00,LOW_FITNESS,PASS,NaN,NaN,2.0600,NaN
2,n2R8WAa,NaN,0.01,LOW_TURNOVER,PASS,NaN,NaN,0.2167,NaN
3,n2R8WAa,NaN,0.70,HIGH_TURNOVER,PASS,NaN,NaN,0.2167,NaN
4,n2R8WAa,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN


In [31]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:['P3aO1oM' '3RreA26']
Passed alphas:['GLAegPJ', '3RrejMz', 'JOEvpbx', '8Q9evom', '3RrejbN', 'XnQ8pka', 'd0VRKA2', 'j0Rrebo', 'VPj38NG', 'EEWeKMr', 'rlgPoWj', '0E3Mj3v', 'n2R8WAa', '9r17EqK', 'j0RrezQ', 'APZ738Y']


# Temp

16th Sep submit it!

In [18]:
len(passed_alphas)

48

In [19]:
#calling submit_alpha on all alphas that have passed the submission tests
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep15_pass_AGLB'])

In [10]:
submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas[10:]}
submit_result

{'L1j3KZ6': False,
 '2L0exAw': False,
 '1d9299R': False,
 '9rv2a7K': False,
 'e0pW9l6': False,
 'anYgEPw': False,
 'anYg75R': False,
 'EE7202R': False,
 'QVJoV2w': False,
 'wlmvp7d': False,
 'zmMLPRK': False,
 'j0EM32k': False,
 'EE723Gr': False,
 'k079aOz': False,
 'XnrMOMm': False,
 'xk65v5n': False,
 'j0EM275': False,
 'L1j2g1e': False,
 '5ObYEez': False,
 'JOwrrGE': False,
 '1d92aOJ': False,
 'llqNlR7': False,
 'EE7xxeR': False,
 'n2JXVjd': False,
 'd0Xv6aX': False,
 'olpZZN6': False,
 'Y0r92lJ': False,
 'APW2J5l': False,
 'OZYgWVp': False,
 'olpkO06': False,
 'QVJoPxQ': False,
 '9rvk2mV': False,
 'anYgvmR': False,
 'anYgY62': False,
 'R83PvP1': False,
 'e0pW0dg': False,
 'VP9R3lw': False,
 'Y0rY19R': False}

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

In [37]:
ace.get_alpha_pnl(s, result[0]['alpha_id'])

,Pnl,alpha_id
Date,,
2012-01-23,0.0,dx9ALqJ
2012-01-24,52190.0,dx9ALqJ
2012-01-25,32038.0,dx9ALqJ
2012-01-26,37403.0,dx9ALqJ
2012-01-27,53631.0,dx9ALqJ
...,...,...
2022-01-17,12935472.0,dx9ALqJ
2022-01-18,12935661.0,dx9ALqJ
2022-01-19,12967474.0,dx9ALqJ
